In this notebook, I will attempt at implementing **Content Based Recommendation System**.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string
import re
import os
%matplotlib inline

In [2]:
pre_df=pd.read_csv("/Users/neirinzaralwin/Developer/personal/product_recommendation_model_api/data/raw/flipkart_sample.csv", na_values=["No rating available"])

In [3]:
pre_df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,NaN,NaN,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,NaN,NaN,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,NaN,NaN,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,NaN,NaN,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,NaN,NaN,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [4]:
pre_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  bool   
 10  description              19998 non-null  object 
 11  product_rating           1849 non-null   float64
 12  overall_rating           1849 non-null   float64
 13  brand                    14136 non-null  object 
 14  product_specifications

## ***Data Preprocessing***

In [ ]:
pre_df['product_category_tree']=pre_df['product_category_tree'].map(lambda x:x.strip('[]'))
pre_df['product_category_tree']=pre_df['product_category_tree'].map(lambda x:x.strip('"'))
pre_df['product_category_tree']=pre_df['product_category_tree'].map(lambda x:x.split('>>'))

axis=0 → Drop rows.
axis=1 → Drop columns.

In [ ]:
#delete unwanted columns
del_list=['crawl_timestamp','product_url',"retail_price","discounted_price","is_FK_Advantage_product","product_rating","overall_rating","product_specifications"]
pre_df=pre_df.drop(del_list,axis=1)

In [ ]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 
exclude = set(string.punctuation)
import string

In [ ]:
pre_df.head()

In [ ]:
pre_df.shape

In [ ]:
smd=pre_df.copy()
# drop duplicate produts
smd.drop_duplicates(subset ="product_name", 
                     keep = "first", inplace = True)
smd.shape

### Data Cleaning

Let’s say doc = "The cats are running fast!".

- Lowercased: "the cats are running fast!"
- Removed stopwords: "cats running fast!"
- Removed punctuation: "cats running fast"
- Tokenized: ["cats", "running", "fast"]
- Lemmatized: ["cat", "run", "fast"]

In [ ]:
def filter_keywords(doc):
    doc=doc.lower()
    stop_free = " ".join([i for i in doc.split() if i not in stop_words])
    punc_free = "".join(ch for ch in stop_free if ch not in exclude)
    word_tokens = word_tokenize(punc_free)
    filtered_sentence = [(lem.lemmatize(w, "v")) for w in word_tokens]
    return filtered_sentence

In [ ]:

smd['product'] = smd['product_name'].apply(filter_keywords)
smd['description'] = smd['description'].astype("str").apply(filter_keywords)
smd['brand'] = smd['brand'].astype("str").apply(filter_keywords)

In [ ]:
smd["all_meta"]=smd['product']+smd['brand']+ pre_df['product_category_tree']+smd['description']
smd["all_meta"] = smd["all_meta"].apply(lambda x: ' '.join(x))

In [ ]:
smd["all_meta"].head()

<strong>TfidfVectorizer</strong> will:

- Tokenize the text into words and bigrams.
- Remove stopwords.
- Compute the Term Frequency-Inverse Document Frequency (TF-IDF) score for each token.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=1, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['all_meta'])

### Cosine Similarity
I will be using the Cosine Similarity to calculate a numeric quantity that denotes the similarity between two products.
Since we have used the TF-IDF Vectorizer, calculating the Dot Product will directly give us the Cosine Similarity Score.

<strong>Example Use Case:</strong>
Suppose you're building a document recommendation system.

- Compute the cosine similarity between all documents in your dataset.
- Use the similarity scores to recommend the most similar documents to a given document

In [ ]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

We now have a pairwise cosine similarity matrix for all the products in our dataset. The next step is to write a function that returns the most similar products based on the cosine similarity score.

In [ ]:
def get_recommendations(title):
    # Step 1: Find the index of the given product/document by its title
    idx = indices[title]
    
    # Step 2: Get the cosine similarity scores of the given product/document with all other products/documents
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Step 3: Sort the similarity scores in descending order (most similar to least similar)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Step 4: Select the top 30 most similar products/documents (excluding the first one, which is the document itself)
    sim_scores = sim_scores[1:31]
    
    # Step 5: Extract the indices of the recommended products/documents
    product_indices = [i[0] for i in sim_scores]
    
    # Step 6: Return the titles (or names) of the recommended products/documents using the extracted indices
    return titles.iloc[product_indices]


In [ ]:
smd = smd.reset_index()
titles = smd['product_name']
indices = pd.Series(smd.index, index=smd['product_name'])

Let us now try and get the top recommendations for a few products.

In [ ]:
get_recommendations("FabHomeDecor Fabric Double Sofa Bed").head(5)

In [ ]:
get_recommendations("Alisha Solid Women's Cycling Shorts").head(5)

In [ ]:
# get_recommendations("Alisha Solid Women's Cycling Shorts").head(5).to_csv("Alisha Solid Women's Cycling Shorts recommendations",index=False,header=True)